In [22]:
import pickle
import streamlit as st
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
file_name= 'finalized_model.sav'
print(file_name)

classifier = pickle.load(open(file_name, 'rb'))
print(classifier.predict([[0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1]]))









finalized_model.sav
[0]


In [16]:
!pip install -q pyngrok
!pip install -q streamlit

!pip install -q streamlit_ace








In [43]:

%%writefile app.py
 
import pickle
import streamlit as st
from sklearn.preprocessing import StandardScaler
 
# loading the trained model
file_name= 'finalized_model.sav'

classifier = pickle.load(open(file_name, 'rb'))
#pickle_in = open('classifier.pkl', 'rb') 
#classifier = pickle.load(pickle_in)
import hashlib

 
@st.cache()
#passlib,hashlib,bcrypt,scrypt

def make_hashes(password):
    return hashlib.sha256(str.encode(password)).hexdigest()

def check_hashes(password,hashed_text):
    if make_hashes(password) == hashed_text:
        return hashed_text
    else:
        print('try again')
# DB Management
import sqlite3 
conn = sqlite3.connect('data.db')
c = conn.cursor()
# DB  Functions
def create_usertable():
    c.execute('CREATE TABLE IF NOT EXISTS userstable(username TEXT,password TEXT)')


def add_userdata(username,password):
    c.execute('INSERT INTO userstable(username,password) VALUES (?,?)',(username,password))
    conn.commit()

def login_user(username,password):
    c.execute('SELECT * FROM userstable WHERE username =? AND password = ?',(username,password))
    data = c.fetchall()
    return data


def view_all_users():
    c.execute('SELECT * FROM userstable')
    data = c.fetchall()
    return data
  
# defining the function which will make the prediction using the data which the user inputs 
def prediction(Fever,Tiredness,Dry_cough,Difficulty_in_Breathing,Sore_Throat,Pains,Nasal_Congestion,Runny_Nose, Diarrhea,Age_0_9, Age_10_19,Age_20_24,Age_25_59,Age_60_Plus,Gender_Female,Gender_Male):
    
    

    
    scaler = StandardScaler()
    Fever = 0 if Fever == 'No' else 1
    Tiredness=0 if Tiredness=='No' else 1
    Dry_cough=0 if Dry_cough=='No' else 1
    Difficulty_in_Breathing=0 if Difficulty_in_Breathing=='No' else 1
    Sore_Throat=0 if Sore_Throat=='No' else 1
    Pains=0 if Pains=='No' else 1
    Nasal_Congestion=0 if Nasal_Congestion == 'No' else 1
    Runny_Nose=0 if Runny_Nose =='No' else 1
    Diarrhea=0 if Diarrhea=='No'else 1


    
    if Age_0_9 == 'Yes':
        
        Age_0_9 = 1
    elif Age_0_9=='No':
        Age_0_9=0
    
    if Age_10_19=='Yes':
        
        Age_10_19=1
    elif Age_10_19=='No':
        Age_10_19=0
    
    
    if Age_20_24 == 'Yes':
        
        Age_20_24 = 1
    elif Age_20_24 == 'No':
        Age_20_24=0
    
    if Age_25_59 == 'Yes':
        
        Age_25_59= 1
    elif Age_25_59 == 'No':
        Age_25_59=0
    if Age_60_Plus== 'Yes':
        
        Age_60_Plus=1
    elif Age_60_Plus=='No':
        Age_60_Plus=0
    if Gender_Female=='Yes':
        
        Gender_Female=1
    elif Gender_Female=='No':
        Gender_Female=0
    
    if Gender_Male=='Yes':
        
        Gender_Male=1
    elif Gender_Male=='No':
        Gender_Male=0
    




    

    input_data = scaler.fit_transform([[Fever,Tiredness,Dry_cough,Difficulty_in_Breathing,Sore_Throat,Pains,Nasal_Congestion,Runny_Nose, Diarrhea,Age_0_9, Age_10_19,Age_20_24,Age_25_59,Age_60_Plus,Gender_Female,Gender_Male ]])
    prediction = classifier.predict(input_data)
    predict_probability = classifier.predict_proba(input_data)
    if prediction == 0:
        pred=0
    else:
        pred=prediction
    return pred
        

# this is the main function in which we define our webpage  
def main():       
    # front end elements of the web page 
    html_temp = """ 
    <div style ="background-color:yellow;padding:13px"> 
    <h1 style ="color:black;text-align:center;">Covid_19 Prediction ML App</h1> 
    </div> 
    """
    #signupand sign in
    """Simple Login App"""

    st.title("User inter Face")

    menu = ["Home","Login","SignUp"]
    choice = st.sidebar.selectbox("Menu",menu)

    if choice == "Home":
        st.subheader("Home")

    elif choice == "Login":
        st.subheader("Login Section")
        username = st.sidebar.text_input("User Name")
        password = st.sidebar.text_input("Password",type='password')
        
        if st.sidebar.checkbox("Login"):
            create_usertable()
            hashed_pswd = make_hashes(password)
            result = login_user(username,check_hashes(password,hashed_pswd))
            if result:
                st.success("Logged In as {}".format(username))
                task = st.selectbox("Task",options=["Add Post","Analytics","Profiles"])
                if task == "Add Post":
                    st.subheader("Add Your Post")
                elif task == "Analytics":
                    st.subheader("Analytics")
                elif task == "Profiles":
                    st.subheader("User Profiles")
                    user_result = view_all_users()
                    clean_db = pd.DataFrame(user_result,columns=["Username","Password"])
                    st.dataframe(clean_db)
                else:
                    st.warning("Incorrect Username/Password")
    elif choice == "SignUp":
            st.subheader("Create New Account")
            new_user = st.text_input("Username")
            new_password = st.text_input("Password",type='password')
            if st.button("Signup"):
                create_usertable()
                add_userdata(new_user,make_hashes(new_password))
                st.success("You have successfully created a valid Account")
                st.info("Go to Login Menu to login")
        
    
    
    st.markdown(html_temp, unsafe_allow_html = True) 
      
    # following lines create boxes in which user can enter data required to make prediction 
    st.title("check whether you covid19 tested positive or not")
    Fever = st.selectbox('Fever',options=['Yes','No'])


    Tiredness=st.selectbox("Tiredness", options=['Yes', 'No'])
    Dry_cough=st.selectbox("Dry_cough",options=['Yes', 'No'])
    Difficulty_in_Breathing=st.selectbox('Difficulty_in_Breathing', options=['Yes','No'])
    Sore_Throat=st.selectbox('Sore_Throat',options=['Yes','No'])
    Pains=st.selectbox('Pains', options=['Yes','No'])

    Nasal_Congestion=st.selectbox('Nasal_Congestion', options=['Yes','No'])
    Runny_Nose=st.selectbox('Runny_Nose', options=['Yes','No'])
    Diarrhea=st.selectbox('Diarrhea', options=['Yes','No'])
    Age_0_9=st.selectbox('Age_0_9', options=['Yes','No'])
    Age_10_19=st.selectbox('Age_10_19',options=['Yes','No'])
    Age_20_24=st.selectbox('Age_20_24',options=['Yes','No'])
    Age_25_59=st.selectbox('Age_25_59',options=['Yes','No'])
    Age_60_Plus =st.selectbox('Age_60_Plus',options=['Yes','No'])
    Gender_Female=st.selectbox('Gender_female',options=['Yes','No'])
    Gender_Male =st.selectbox('Gender_Male',options=['Yes','No'])
    result =""
      
    # when 'Predict' is clicked, make the prediction and store it 
    if st.button("Predict"): 
        result = prediction(Fever,Tiredness,Dry_cough,Difficulty_in_Breathing,Sore_Throat,Pains,Nasal_Congestion,Runny_Nose, Diarrhea,Age_0_9, Age_10_19,Age_20_24,Age_25_59,Age_60_Plus,Gender_Female,Gender_Male) 
        st.success('the probability that you are covid19 test postive {}'.format(result))
        print('the accrucy of the model at max 50%')
if __name__=='__main__': 
    main()

Overwriting app.py
